In [0]:
catlog_name = 'harshit_gupta_databricks_npmentorskool_onmicrosoft_com'
schema_name = 'gbmartecoschema'
mount_point = "/mnt/mskltestmnt2643"

files = dbutils.fs.ls(mount_point)
display(files)

In [0]:
#create a list of all the tables name
tables = []

for file in files:
    file_name = file.name.split('.')[0]
    tables.append(file_name)
print(tables)

In [0]:
# create the schema
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {catlog_name}.{schema_name}")

In [0]:
# create the tables
for table in tables:
    spark.sql(f"CREATE TABLE IF NOT EXISTS {catlog_name}.{schema_name}.{table}")

In [0]:
%sql
show tables in harshit_gupta_databricks_npmentorskool_onmicrosoft_com.gbmartecoschema

In [0]:
# Enable column mapping
spark.sql("SET spark.databricks.delta.columnMapping.mode = name")
spark.sql("SET spark.databricks.delta.schema.autoMerge.enabled = true")

# Now we want to load the data into tables by copy into the tables
for table in tables:
    spark.sql(f"""
        COPY INTO {catlog_name}.{schema_name}.{table}
        FROM '{mount_point}/{table}.csv'
        FILEFORMAT = CSV
        FORMAT_OPTIONS (
            'header' = 'true',
            'inferSchema' = 'true',
            'timestampFormat' = 'dd-MM-yyyy HH.mm',
            'multiline' = 'true',
            'escape' = '"',
            'quote' = '"',
            'mergeSchema' = 'true'
        )
        COPY_OPTIONS ('mergeSchema' = 'true')
    """)

In [0]:
# fetch the data from addresses table
addresses = spark.sql(f"SELECT * FROM {catlog_name}.{schema_name}.addresses")
display(addresses)

In [0]:
customers = spark.sql(f"SELECT * FROM {catlog_name}.{schema_name}.customers")
display(customers)

In [0]:
%sql
ALTER TABLE harshit_gupta_databricks_npmentorskool_onmicrosoft_com.gbmartecoschema.orders SET TBLPROPERTIES (delta.enableChangeDataFeed = true)